In [1]:
import pandas as pd
import sys
import os
from os import listdir
os.chdir("c:/Users/Robert/Documents/Projekte/dev/sport_betting/")
import config as CONFIG
pd.set_option('display.max_columns', 500)

from src.features import load_team_opponent
from src.features.preprocess import derived_odds,get_bookmaker, get_odd_pred, split_date,add_last_3_scores_column
import numpy as np

# Lade Daten ein

## Team Opponent

In [2]:
df = load_team_opponent(filename_main = CONFIG.DATA_FOLDER_RAW+'main_leagues_database_raw.sav')


C:\Users\Robert\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  """Entry point for launching an IPython kernel.


In [3]:
sum(df.B365_Opponent.isna())

4

## Sortiere den Datensatz nach Datum, für die Durchschnittsfelder

In [4]:
df = split_date(df)
df = df.sort_values(['year','month','day'],ascending=True)
df = df.reset_index()
df.Date = pd.to_datetime(df.Date)

In [5]:
df["diff_FTG_against_Opponent"] = df["FTG_Team"]-df["FTG_Opponent"]
df["diff_ShotsOnTarget_against_Opponent"] = df["ShotsOnTarget_Team"]-df["ShotsOnTarget_Opponent"]

# Erstelle Zielvariable

In [6]:
df['Target'] = df.FTG_Team>df.FTG_Opponent
df['Target'].value_counts()

False    8762
True     5264
Name: Target, dtype: int64

# Features entwicklen

In [7]:
standad_attributes = ['Target','atHome','Date','Div']
bookmaker_attributes = get_bookmaker(bm = 'BW',df = df)+get_bookmaker(bm='B365',df = df)+get_bookmaker(bm='IW',df = df)
bet_on_team = get_bookmaker(bm='Team',df=df[bookmaker_attributes])
bet_on_opponent = get_bookmaker(bm='Opponent',df=df[bookmaker_attributes])
bet_on_draw = get_bookmaker(bm='Draw',df=df[bookmaker_attributes])


In [8]:
df_derived_b365 = get_odd_pred(bet="B365",df=df)
df_derived_iw = get_odd_pred(bet="IW",df=df)
df_derived_bw = get_odd_pred(bet="BW",df=df)


In [9]:
df_derived_team = derived_odds(sight = 'Team',df=df, odds=bet_on_team)
df_derived_opponent = derived_odds(sight = 'Opponent',df=df, odds=bet_on_opponent)
df_derived_draw = derived_odds(sight = 'Draw',df=df, odds=bet_on_draw)


In [10]:
# df = add_last_3_scores_column(df = df, score_column="diff_FTG_against_Opponent",anz_games=4)
# df = add_last_3_scores_column(df = df, score_column="diff_ShotsOnTarget_against_Opponent",anz_games=4)
# df = add_last_3_scores_column(df = df, score_column="B365_Team",anz_games=4)
# df = add_last_3_scores_column(df = df, score_column="B365_Opponent",anz_games=4)
# df = df.dropna()
# rolling_mean_attributes = [c for c in  df.columns if 'last' in c]

# Modelldatensatz erstellen

In [11]:
model_data = pd.concat([df[standad_attributes],
                        #df[rolling_mean_attributes],
                        df[bookmaker_attributes],
                        df[["Team","Opponent","month","year"]],
                        df_derived_team,
                        df_derived_opponent,
                        df_derived_draw,
                        df_derived_b365,
                        df_derived_iw,
                        df_derived_bw],axis=1)

In [12]:
model_data.replace([np.inf, -np.inf], np.nan, inplace=True)

#model_data = model_data.dropna()

In [13]:
df.IW_Team.isna().sum()

30

## Encoding Teams

In [14]:
from sklearn import preprocessing
le_div = preprocessing.LabelEncoder()
le_team = preprocessing.LabelEncoder()


In [15]:
le_team.fit(list(set(model_data.Team)))
model_data.Team = le_team.transform(model_data.Team)
model_data.Opponent = le_team.transform(model_data.Opponent)

In [16]:
le_div.fit(list(set(model_data.Div)))
model_data.Div = le_div.transform(model_data.Div)


Aufteilen in Training, Test und Validierung

In [17]:
model_data.Date = pd.to_datetime(model_data.Date)

In [18]:
import datetime
datum_1_1_2020 = datetime.datetime(2021, 6, 1)
test_data = model_data[model_data.Date>datum_1_1_2020]
model_data = model_data[model_data.Date<datum_1_1_2020]


In [19]:
datum_1_1_2020 = datetime.datetime(2019, 6, 1)
model_data_valid = model_data[model_data.Date>datum_1_1_2020]
model_data_train = model_data[model_data.Date<datum_1_1_2020]

In [20]:
len(model_data_train)

9472

In [21]:
model_data_train.to_csv(CONFIG.DATA_FOLDER_PROCESSED + 'Train/model_data.csv')
model_data_valid.to_csv(CONFIG.DATA_FOLDER_PROCESSED + 'Valid/model_data.csv')
test_data.to_csv(CONFIG.DATA_FOLDER_PROCESSED + 'Test/model_data.csv')
